<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow2/Tutorial/Getting_started_TensorFlow2/Validation_regularisation_callbacks/Additional_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile pip_install.sh
#!/bin/bash
pip install -qU pip wheel
pip install -qU numpy pandas matplotlib
pip install -qU scikit-learn

# Additional callbacks

In this reading we'll be looking at more of the inbuilt callbacks available in Keras.

In [1]:
import tensorflow as tf
print(tf.__version__)

2.9.1


We will again be using the sklearn diabetes dataset to demonstrate these callbacks.

In [2]:
# Load the diabetes dataset
from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()

In [3]:
# Save the input and target variables
data = diabetes_dataset['data']
targets = diabetes_dataset['target']

In [4]:
# Split the data set into training and test sets
from sklearn.model_selection import train_test_split

train_data, test_data, train_targets, test_targets = train_test_split(
    data, targets, test_size=0.1)

Let's also build a simple model to fit the data with our callbacks.

In [6]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

# Print the model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               1408      
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 18,049
Trainable params: 18,049
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# Compile the model
model.compile(
    loss='mse', 
    optimizer='adam', 
    metrics=['mse', 'mae']
)

Now onto the callbacks!

## Learning rate scheduler

**usage:** `tf.keras.callbacks.LearningRateScheduler(schedule, verbose=0)`

The learning rate scheduler that we implemented in the previous reading as a custom callback is also available as a built in callback.

As in our custom callback, the `LearningRateScheduler` in Keras takes a function `schedule` as an argument.

This function `schedule` should take two arguments:
* The current epoch (as an integer), and
* The current learning rate,

and return new learning rate for that epoch.

The `LearningRateScheduler` also has an optional `verbose` argument, which prints information about the learning rate if it is set to 1.

Let's see a simple example.

In [8]:
# Define the learning rate schedule function
def lr_function(epoch, lr):
    if epoch%2==0:
        return lr
    else:
        return lr + epoch/1000

In [9]:
# Train the model
history = model.fit(train_data, train_targets, epochs=10,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_function, 
    verbose=1)], verbose=False)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.

Epoch 2: LearningRateScheduler setting learning rate to 0.0020000000474974513.

Epoch 3: LearningRateScheduler setting learning rate to 0.0020000000949949026.

Epoch 4: LearningRateScheduler setting learning rate to 0.005000000094994903.

Epoch 5: LearningRateScheduler setting learning rate to 0.004999999888241291.

Epoch 6: LearningRateScheduler setting learning rate to 0.009999999888241292.

Epoch 7: LearningRateScheduler setting learning rate to 0.009999999776482582.

Epoch 8: LearningRateScheduler setting learning rate to 0.01699999977648258.

Epoch 9: LearningRateScheduler setting learning rate to 0.016999999061226845.

Epoch 10: LearningRateScheduler setting learning rate to 0.025999999061226846.


You can also use lambda functions to define your schedule given an epoch.

In [12]:
# Train the model with a difference schedule
history = model.fit(train_data, train_targets, epochs=10,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(
        lambda x:1/(3+5*x), verbose=1)], verbose=False)


Epoch 1: LearningRateScheduler setting learning rate to 0.3333333333333333.

Epoch 2: LearningRateScheduler setting learning rate to 0.125.

Epoch 3: LearningRateScheduler setting learning rate to 0.07692307692307693.

Epoch 4: LearningRateScheduler setting learning rate to 0.05555555555555555.

Epoch 5: LearningRateScheduler setting learning rate to 0.043478260869565216.

Epoch 6: LearningRateScheduler setting learning rate to 0.03571428571428571.

Epoch 7: LearningRateScheduler setting learning rate to 0.030303030303030304.

Epoch 8: LearningRateScheduler setting learning rate to 0.02631578947368421.

Epoch 9: LearningRateScheduler setting learning rate to 0.023255813953488372.

Epoch 10: LearningRateScheduler setting learning rate to 0.020833333333333332.


## CSV logger
**Usage** `tf.keras.callbacks.CSVLogger(filename, separator=',', append=False)`

This callback streams the results from each epoch into a CSV file. The first line of the CSV file will be the name of pieces of information recorded on each subsequent line, begining with the epoch and loss value. The values of metrics at the end of each epoch will also be recorded.

The only compulsory argument is the `filename` for the log to be streamed to. This could also be a filepath.

You can also specify the `separator` to be used between entries on each line.

The `append` argument allows you the option to append your results to an existing file with the same name. This can be particularly useful if you are continuing training.

Let's see an example.

In [15]:
# Train the model with a CSV logger
history = model.fit(train_data, train_targets, epochs=10,
    callbacks=[tf.keras.callbacks.CSVLogger("results.csv")], 
    verbose=False)

Let's view the information in the CSV file we have created using `pandas`.

In [16]:
# Load the CSV
import pandas as pd

pd.read_csv("results.csv", index_col='epoch')

,loss,mae,mse
epoch,,,
0,29458.884766,152.967911,29458.884766
1,29221.042969,152.201889,29221.042969
2,28222.654297,148.932053,28222.654297
3,24383.640625,135.874237,24383.640625
4,14581.215820,96.422249,14581.215820
5,5343.618164,60.887852,5343.618164
6,4209.692383,55.968082,4209.692383
7,3906.019287,51.729214,3906.019287
8,3479.827148,49.646446,3479.827148


## Lambda callbacks
**Usage** `tf.keras.callbacks.LambdaCallback(                
        on_epoch_begin=None, on_epoch_end=None,                 
        on_batch_begin=None, on_batch_end=None,                 
        on_train_begin=None, on_train_end=None)`
    
Lambda callbacks are used to quickly define simple custom callbacks with the use of lambda functions.

Each of the functions require some positional arguments.
* `on_epoch_begin` and `on_epoch_end` expect two arguments: `epoch` and `logs`,
* `on_batch_begin` and `on_batch_end` expect two arguments: `batch` and `logs` and 
* `on_train_begin` and `on_train_end` expect one argument: `logs`.

Let's see an example of this in practice.

In [19]:
# Print the epoch number at the beginning of each epoch
epoch_callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_begin=lambda epoch, logs: print('Starting Epoch {}!'.format(epoch+1)))

In [20]:
# Print the loss at the end of each batch
batch_loss_callback = tf.keras.callbacks.LambdaCallback(
    on_batch_end=lambda batch, logs: print('\n After batch {}, the loss is {:7.2f}.'
    .format(batch, logs['loss'])))

In [21]:
# Inform that training is finished
train_finish_callback = tf.keras.callbacks.LambdaCallback(
    on_train_end=lambda logs: print('Training finished'))

In [22]:
# Train the model with the lambda callbacks
history = model.fit(train_data, train_targets, epochs=5, batch_size=100,
    callbacks=[epoch_callback, batch_loss_callback, train_finish_callback], 
    verbose=False)

Starting Epoch 1!

 After batch 0, the loss is 31990.04.

 After batch 1, the loss is 29193.77.

 After batch 2, the loss is 28590.04.

 After batch 3, the loss is 29491.18.
Starting Epoch 2!

 After batch 0, the loss is 26294.07.

 After batch 1, the loss is 28054.60.

 After batch 2, the loss is 27926.27.

 After batch 3, the loss is 29474.50.
Starting Epoch 3!

 After batch 0, the loss is 26292.26.

 After batch 1, the loss is 26909.12.

 After batch 2, the loss is 28392.31.

 After batch 3, the loss is 29450.07.
Starting Epoch 4!

 After batch 0, the loss is 32103.93.

 After batch 1, the loss is 30001.99.

 After batch 2, the loss is 28186.84.

 After batch 3, the loss is 29413.72.
Starting Epoch 5!

 After batch 0, the loss is 29753.60.

 After batch 1, the loss is 28771.25.

 After batch 2, the loss is 29399.19.

 After batch 3, the loss is 29359.00.
Training finished


## Reduce learning rate on plateau
**Usage** `tf.keras.callbacks.ReduceLROnPlateau(                        
            monitor='val_loss', factor=0.1, patience=10, verbose=0,            
            mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)` 
    
The `ReduceLROnPlateau` callback allows reduction of the learning rate when a metric has stopped improving.
The arguments are similar to those used in the `EarlyStopping` callback.
* The argument `monitor` is used to specify which metric to base the callback on.
* The `factor` is the factor by which the learning rate decreases i.e., new_lr=factor*old_lr.
* The `patience` is the number of epochs where there is no improvement on the monitored metric before the learning rate is reduced.
* The `verbose` argument will produce progress messages when set to 1.
* The `mode` determines whether the learning rate will decrease when the monitored quantity stops increasing (`max`) or decreasing (`min`). The `auto` setting causes the callback to infer the mode from the monitored quantity.
* The `min_delta` is the smallest change in the monitored quantity to be deemed an improvement.
* The `cooldown` is the number of epochs to wait after the learning rate is changed before the callback resumes normal operation.
* The `min_lr` is a lower bound on the learning rate that the callback will produce.

Let's examine a final example.

In [8]:
# Train the model with the ReduceLROnPlateau callback
history = model.fit(train_data, train_targets, epochs=100, batch_size=100,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(
        monitor='loss', factor=0.2, verbose=1)], verbose=False)

### Further reading and resources

* https://keras.io/callbacks/
* https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LearningRateScheduler
* https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/CSVLogger
* https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LambdaCallback